# Meta-data generation script for Microson enhancement dataset (V1)

Perse Multilingual-LibriSpeech-Spanish (MLLSS) and WHAM! noise datasets and define situations and augmentations for each MLLSS instance

In [1]:
import masp as srs
import numpy as np
import soundfile as sf
from IPython.display import Audio
import scipy
import copy
import pandas as pd
import os
from os.path import join as pjoin
from multiprocessing import Pool
import matplotlib.pyplot as plt
import mat73
import tqdm
import pyrubberband as pyrb


In [2]:
# import my modules (helpers.py where I stored all the functions):
import helpers as hlp
import importlib
importlib.reload(hlp);

In [3]:
# WE DEFINE THE CHUNK LENGHT TO 4 SECONDS (DEFAULT TO USING SUDO-RM-RF)
# SET PATHS:
mls_path = '/home/ubuntu/Data/mls_spanish'
wham_path = '/home/ubuntu/Data/wham'
output_path = '/home/ubuntu/Data/ha_scenes_sounds/'
fs = 16000
fs_n = 'wav16k'
mode = 'min'

In [4]:
np.random.seed(0)

In [5]:
def assign_chunks_by_duration(seconds_list):
    # Files that can reasonably be padded will be padded.
    # Otherwise we crop
    out = []
    chunk = []
    low_bound = np.array([(x+1)*2 for x in list(range(11))])   
    hi_bound = np.array([(x+2)*2 for x in list(range(11))])
    n_chunks = np.repeat(np.array([x+1 for x in list(range(6))]), 2)
    n_chunks = n_chunks[:len(low_bound)]
    for s in seconds_list:
        x = n_chunks[np.argmax(np.logical_and(s > low_bound, s < hi_bound))]
        out.append(x)
        chunk.append(list(range(x)))
    return out, chunk   

In [6]:
'''
noise_info = []
for split in ['tr', 'cv', 'tt']:
    split_path = pjoin(pjoin(wham_path, 'wham_noise'), split)
    for audio_path in  tqdm.tqdm(os.listdir(split_path)):
        audio, fs = sf.read(pjoin(split_path, audio_path))
        noise_info.append({'split': split, 'audio_path': pjoin(audio_path), 'fs_noise': fs, 'shape':audio.shape})
        

ometa = pd.read_csv(pjoin(mls_path, 'metainfo.txt'), sep= '|')
ometa.columns = ometa.columns.str.strip()
ometa['PARTITION'] = ometa['PARTITION'].str.strip()

info = []
for split in ['train', 'test', 'dev']:
    split_path = pjoin(pjoin(mls_path, split), 'audio')
    speakers = os.listdir(split_path)
    for speaker in speakers:
        speaker_path = pjoin(split_path, speaker)
        books = os.listdir(speaker_path)
        for book in books:
            book_path = pjoin(speaker_path, book)
            audio_paths = os.listdir(book_path)
            for audio_path in tqdm.tqdm(audio_paths):
                gender = list(ometa[ometa['SPEAKER']==int(speaker)]['GENDER'])[0].strip()
                info.append({'split': split, 'speaker': speaker, 'book': book, 'speech_path': audio_path, 'gender': gender})
df_s = pd.DataFrame(info)
df_n = pd.DataFrame(noise_info)


lens = list(df_n['shape'])

df_s.to_pickle("mls_info.pkl")
df_n.to_pickle("wham_info.pkl")
''';

In [7]:
df_s = pd.read_pickle("mls_info.pkl")
df_n = pd.read_pickle("wham_info.pkl")

In [8]:
def augment_wham_train_set(tr, speech_len, fs):
    tr = tr.reset_index(drop=True)
    # Store lenght (in samples and seconds) from shape
    tr.insert(2, "len_samp", [x[0] for x in list(tr['shape'])], True)
    tr.insert(2, "len_s", [x[0]/fs for x in list(tr['shape'])], True)
    
    # Assign a number of chunks. Chunks from [2,4]s are expanded to one 4s chunk. 
    # Chunks from [4,6]s are cropped to one 4s chunk
    nchunks, chunk = assign_chunks_by_duration(list(tr['len_s']))
    tr.insert(2, "num_chunks", nchunks, True)
    lentr = len(tr)
    tr = tr.reindex(tr.index.repeat(tr.num_chunks))
    tr = tr.reset_index(drop=True)
    tr.insert(2, "chunk", [item for sublist in chunk for item in sublist], True)
    # One copy will have phase inversion (*(-1))
    L = [False, True]
    tr = (pd.DataFrame(np.repeat(tr.values, 2, axis=0), columns=tr.columns)
               .assign(phase_inv = np.tile(L, len(tr))))
    # The other copy will have a swap of left and right channels
    tr = (pd.DataFrame(np.repeat(tr.values, 2, axis=0), columns=tr.columns)
               .assign(lr_inv = np.tile(L, len(tr))))
    # The rest of utterances we have to augment will randomly time-streched
    stretch_utt = speech_len - len(tr)
    stretch  = np.concatenate((np.zeros(len(tr)), np.random.uniform(low=0.9, high=1.1, size=(stretch_utt))))
    tr = pd.concat([tr, tr[0:stretch_utt]])
    tr.insert(10, "stretch", stretch, True)
    tr = tr.reset_index(drop=True)
    tr = tr.rename(columns={'audio_path': 'noise_path'})
    tr = tr.rename(columns={'split': 'wham_split'})
    return tr

def crop_wham_test_set(tr, speech_len, fs):
    tr = tr.reset_index(drop=True)
    tr.insert(2, "len_samp", [x[0] for x in list(tr['shape'])], True)
    tr.insert(2, "len_s", [x[0]/fs for x in list(tr['shape'])], True)
    tr = tr[tr.len_s > 4.]
    tr = tr[:speech_len]
    tr = tr.reset_index(drop=True)
    tr.insert(2, "chunk", [0]*len(tr))
    tr.insert(2, "num_chunks", [1]*len(tr))
    tr.insert(2, "phase_inv", [False]*len(tr))
    tr.insert(2, "lr_inv", [False]*len(tr))
    tr.insert(2, "stretch", np.zeros(len(tr)))
    tr = tr.rename(columns={'audio_path': 'noise_path'})
    tr = tr.rename(columns={'split': 'wham_split'})
    return tr

In [9]:
tr = augment_wham_train_set(df_n[df_n['split']=='tr'], len(df_s[df_s['split']=='train']), fs)
cv = crop_wham_test_set(df_n[df_n['split']=='cv'], len(df_s[df_s['split']=='dev']), fs)
tt = crop_wham_test_set(df_n[df_n['split']=='tt'], len(df_s[df_s['split']=='test']), fs)

In [10]:
aug_wham = pd.concat([tr, tt, cv], axis=0)
aug_wham = aug_wham.reset_index(drop=True)

In [11]:
# now we merge the augmented wham metadata with the MLS-spanish metadata
df = pd.concat([df_s, aug_wham], axis=1)

In [12]:
# import my modules (helpers.py where I stored all the functions):
import helpers as hlp
import importlib
importlib.reload(hlp);

head_orient_azi = np.random.uniform(low = -45, high = 45, size = len(df))
head_orient_ele = np.random.uniform(low = -10, high = 10, size = len(df))

angle = np.random.uniform(low = -45, high = 45, size = len(df))
dist = np.random.uniform(low = 0.5, high = 3, size = len(df))
snr = np.random.uniform(low = 0, high = 6, size = len(df))

In [13]:
room_x = np.random.uniform(low = 3., high = 30., size = len(df))

In [14]:
room_y = room_x * np.random.uniform(low=0.5, high=1, size=len(room_x)) #avoid tunnels

In [15]:
#room_y = np.random.uniform(low = 2., high = 30., size = len(df))
room_z = np.random.uniform(low = 2.5, high = 5., size = len(df))

In [16]:
t60s =  np.random.uniform(low = .1, high = 1., size = len(df))
t60s = np.sort(t60s)
volumes = room_x * room_y * room_z
volumes = np.sort(volumes)
dist = np.sort(dist)
perm = np.random.permutation(len(volumes))
room_x = room_x[perm]
room_y = room_y[perm]
room_z = room_z[perm]
dist = dist[perm]
t60s = t60s[perm]

In [17]:
head_pos = []
for k in range(len(room_x)):
    head_pos.append(np.array([np.random.uniform(low = 0.35*room_x[k], high = 0.65*room_x[k]),
                        np.random.uniform(low = 0.35*room_y[k], high = 0.65*room_y[k]),
                        np.random.uniform(low = 1., high = 2.)]))

In [18]:
head_pos = np.array(head_pos)

In [19]:
room = np.array((room_x, room_y, room_z)).T

In [20]:
# import my modules (helpers.py where I stored all the functions):
import helpers as hlp
import importlib
importlib.reload(hlp);

target_pos = []
for k in tqdm.tqdm(range(len(room_x))):
    target_pos.append(hlp.place_on_circle_in_room(head_pos[k], dist[k], 
                                                               angle[k]+head_orient_azi[k], room[k]))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225494/225494 [00:03<00:00, 69420.93it/s]


In [21]:
target_pos = np.squeeze(np.array(target_pos))

In [22]:
np.all(target_pos > 0.) # all targets are positive

True

In [23]:
# Checks:
np.all(target_pos < room) # all targets are in the room

True

In [24]:
np.all(head_pos < room) # all heads are in the room

True

In [25]:
# now let's check the ears:
ears_pos = []
for k in range(head_pos.shape[0]):
    ears_pos.append(np.array(hlp.head_2_ku_ears(head_pos[k], np.array([head_orient_azi[k],head_orient_ele[k]]))))

ears_pos = np.array(ears_pos)

In [26]:
np.all(ears_pos[:, 0, :] < room) # all left ears are in the room

True

In [27]:
np.all(ears_pos[:, 1, :] < room) # all right are in the room

True

In [28]:
np.all(ears_pos > 0)

True

In [29]:
# final MINIMUM distance between head and target (check we don't have an intra-craneal target)
min(np.sqrt(np.sum((target_pos - head_pos)**2, axis=1))) > 0.0875 * 2

True

In [30]:
# minimum distance of ears against a wall
min ( min(room[:, 0] - ears_pos[:, 0, 0]), min(room[:, 0] - ears_pos[:, 1, 0]))

1.0248253294449863

In [31]:
min ( min(room[:, 1] - ears_pos[:, 0, 1]), min(room[:, 1] - ears_pos[:, 1, 1]))

0.4865705117067447

In [32]:
min ( min(room[:, 2] - ears_pos[:, 0, 2]), min(room[:, 2] - ears_pos[:, 1, 2]))

0.5061991183289214

In [33]:
# minimum distance of targets against a wall
min(min(room[:, 0] - target_pos[:, 0]), min(room[:, 1] - target_pos[:, 1]), min(room[:, 2] - target_pos[:, 2]))

0.20000918012154356

In [34]:
df = df.rename(columns={'split': 'mls_split'})

In [35]:
df

,mls_split,speaker,book,speech_path,gender,wham_split,noise_path,chunk,num_chunks,len_s,len_samp,fs_noise,shape,phase_inv,lr_inv,stretch
0,train,8688,8509,8688_8509_000091.flac,M,tr,012o030z_0.94807_405c020d_-0.94807.wav,0,2,9.290938,148655,16000,"(148655, 2)",False,False,0.0
1,train,8688,8509,8688_8509_000029.flac,M,tr,012o030z_0.94807_405c020d_-0.94807.wav,0,2,9.290938,148655,16000,"(148655, 2)",False,True,0.0
2,train,8688,8509,8688_8509_000069.flac,M,tr,012o030z_0.94807_405c020d_-0.94807.wav,0,2,9.290938,148655,16000,"(148655, 2)",True,False,0.0
3,train,8688,8509,8688_8509_000146.flac,M,tr,012o030z_0.94807_405c020d_-0.94807.wav,0,2,9.290938,148655,16000,"(148655, 2)",True,True,0.0
4,train,8688,8509,8688_8509_000052.flac,M,tr,012o030z_0.94807_405c020d_-0.94807.wav,1,2,9.290938,148655,16000,"(148655, 2)",False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225489,dev,2592,1378,2592_1378_000003.flac,F,cv,01vo030s_0.91624_40nc020b_-0.91624.wav,0,1,10.816313,173061,16000,"(173061, 2)",False,False,0.0
225490,dev,2592,1378,2592_1378_000064.flac,F,cv,40no0304_0.065523_20qa0114_-0.065523.wav,0,1,13.52325,216372,16000,"(216372, 2)",False,False,0.0
225491,dev,2592,1378,2592_1378_000047.flac,F,cv,01ko030u_1.0843_01vo0313_-1.0843.wav,0,1,14.284625,228554,16000,"(228554, 2)",False,False,0.0
225492,dev,2592,1378,2592_1378_000034.flac,F,cv,014c0208_2.3161_028a010r_-2.3161.wav,0,1,6.931,110896,16000,"(110896, 2)",False,False,0.0


In [36]:
df.insert(14, "room_x", room[:, 0])
df.insert(15, "room_y", room[:, 1])
df.insert(16, "room_z", room[:, 2])
df.insert(17, "rt60", t60s)
df.insert(18, "headC_x", head_pos[:,0])
df.insert(19, "headC_y", head_pos[:,1])
df.insert(20, "headC_z", head_pos[:,2])
df.insert(21, "src_x", target_pos[:,0])
df.insert(22, "src_y", target_pos[:,1])
df.insert(23, "src_z", target_pos[:,2])
df.insert(24, "headOrient_azi", head_orient_azi)
df.insert(25, "headOrient_ele", head_orient_ele)
df.insert(26, "snr", snr)

In [37]:
df['idx'] = range(len(df))

In [38]:
'''
# generate a figure for each situation:
for k in tqdm.tqdm(range(head_pos.shape[0])):
    hlp.plot_scene(room[k], head_pos[k], np.array([head_orient_azi[k], head_orient_ele[k]])
                   , ears_pos[k],[target_pos[k]], perspective="xy")
    plt.title(str(head_orient_azi[k])+ '_' + str(angle[k]))
    plt.savefig(pjoin('situation_plots_rot', os.path.splitext(os.path.basename(df.iloc[k].audio_path))[0]+'.pdf'))
    plt.close('all')
''';

In [39]:
df.to_csv('meta_microson_v1.csv', index=False, compression='infer')